# ArcGIS API for Python: AIS Route Extraction Prototype

## A collaboration between the U.S. Department of Transportation and Esri

###### Alberto Nieto (Esri), Andrew Barrows (USDOT), Dominic Menegus (USDOT)

This Jupyter Notebook contains documentation, processing, and data visualization of a prototype process in which the Automated Identification System (AIS) vessel data for specified terminal-to-terminal connections is used to digitize a route polyline dataset for an authoritative GIS. 

This process can be leveraged to develop an Information Product that generates data-driven ferry routes for all terminals in the United States, and could be extended to other environments and use cases.

Contact Info:
anieto@esri.com

# Ask Questions

### How can we determine "average" paths for ferry routes in the United States? 

# Proposed Methodology

#### Use AIS positional data to determine ship positions along routes, perform qualitative checks, and digitize polyline geometry

<img src="../../docs/dbclustering.PNG"></img>

#### Script Set-up (Imports, variables, etc.) (Pre-process)

In [13]:
import arcgis
import plotly
import statsmodels.api as sm
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import norm
%matplotlib inline  

gis = arcgis.gis.GIS()

In [14]:
gis_app_id = r"wt3QUR1M4eum0TVI"
gis_url = r"https://esrifederal.maps.arcgis.com"
gis = arcgis.gis.GIS(gis_url, client_id=gis_app_id)

Please sign in to your GIS and paste the code that is obtained below.
If a web browser does not automatically open, please navigate to the URL below yourself instead.
Opening web browser to navigate to: https://esrifederal.maps.arcgis.com/sharing/rest/oauth2/authorize?client_id=wt3QUR1M4eum0TVI&response_type=code&expiration=-1&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob
Enter code obtained on signing in using SAML: ········


# Part 1: Retrieve and Explore AIS Data

#### Code to Retrieve AIS Data

In [ ]:
ais_sample = gis.content.search("AIS_StatenIsland_Sample", item_type="feature layer")[0]
ais_sample

#### Map of AIS Data

In [ ]:
nyc_map = gis.map("New York City")
nyc_map.add_layer(ais_sample)
nyc_map.basemap = 'streets-night-vector'
nyc_map

<img src="img/capture05.png">

# Part 2: Run Lowess

### Explanation of Loess/Lowess

Locally Weighted Scatterplot (LOWESS) Basic Facts:

- Used in regression analysis to create a smooth line through a scatterplot.

- Has a robust fitting method.  Because of this it can be used when there are noisy data values.

- Non-parametric – no assumptions are made about the shape of the line of fit.

<img src="img/lowess.jpg">

#### Code to run and plot lowess on AIS Staten Island sample (Pre-processed)

In [15]:
def calculate_lat_lon_in_spatialdf(spatialdf, shape_field='SHAPE', output_lat='latitude', output_lon='longitude'):

    def calculate_latitude(shape_field):
        return shape_field['y']

    def calculate_longitude(shape_field):
        return shape_field['x']

    # Calculate latitude and longitude fields from the shape attribute
    spatialdf[output_lat] = spatialdf.apply(lambda x: calculate_latitude(x[shape_field]), axis=1)
    spatialdf[output_lon] = spatialdf.apply(lambda x: calculate_longitude(x[shape_field]), axis=1)

    return spatialdf

In [16]:
ais_local_fgdb = r"D:\ANieto_SolutionEngineer\Data\DOT\BTS\Zone18_2014_07\Zone18_2014_07.gdb"
ais_sample_fc = "{0}//AIS_Sampler".format(ais_local_fgdb)
ais_sdf = calculate_lat_lon_in_spatialdf(arcgis.features.SpatialDataFrame.from_featureclass(ais_sample_fc))
ais_sdf

,OBJECTID,SOG,COG,Heading,ROT,BaseDateTime,Status,VoyageID,MMSI,ReceiverType,ReceiverID,SHAPE,latitude,longitude
0,1,0.0,292.0,243.0,0.0,2014-07-01 00:00:02.000000,0,7,367019000,b,2003669982,"{'x': -74.071757, 'y': 40.64426499999999, 'spa...",40.644265,-74.071757
1,2,0.0,292.0,243.0,0.0,2014-07-01 00:01:02.000000,0,7,367019000,b,2003669982,"{'x': -74.071813, 'y': 40.64412200000001, 'spa...",40.644122,-74.071813
2,3,0.0,292.0,243.0,0.0,2014-07-01 00:02:02.000000,0,7,367019000,b,2003669982,"{'x': -74.071802, 'y': 40.64416299999999, 'spa...",40.644163,-74.071802
3,4,0.0,292.0,243.0,0.0,2014-07-01 00:03:02.000000,0,7,367019000,b,2003669982,"{'x': -74.071812, 'y': 40.64414199999999, 'spa...",40.644142,-74.071812
4,5,0.0,292.0,244.0,0.0,2014-07-01 00:04:02.000000,0,7,367019000,b,2003669982,"{'x': -74.071825, 'y': 40.644149999999996, 'sp...",40.644150,-74.071825
5,6,0.0,292.0,244.0,0.0,2014-07-01 00:05:02.000000,0,7,367019000,b,2003669982,"{'x': -74.07182, 'y': 40.644135000000006, 'spa...",40.644135,-74.071820
6,7,0.0,292.0,244.0,0.0,2014-07-01 00:06:02.000000,0,7,367019000,b,003669984,"{'x': -74.071818, 'y': 40.64414199999999, 'spa...",40.644142,-74.071818
7,8,0.0,292.0,245.0,0.0,2014-07-01 00:07:02.000000,0,7,367019000,b,2003669982,"{'x': -74.071817, 'y': 40.64410000000001, 'spa...",40.644100,-74.071817
8,9,0.0,292.0,245.0,0.0,2014-07-01 00:08:02.000000,0,7,367019000,b,2003669982,"{'x': -74.071768, 'y': 40.644115, 'spatialRefe...",40.644115,-74.071768
9,10,0.0,292.0,245.0,0.0,2014-07-01 00:09:02.000000,0,7,367019000,b,003669984,"{'x': -74.071723, 'y': 40.64408800000001, 'spa...",40.644088,-74.071723


### Plot of First Run of Lowess

In [17]:
plotly.offline.plot({
    "data": [plotly.graph_objs.Scatter(x=ais_sdf['longitude'], y=ais_sdf['latitude'])]
})

'file://C:\\Users\\albe9057\\Documents\\GitHub\\AISFerryRouteDigitization_Prototype\\scripts\\prototype\\temp-plot.html'

<img src="img/newplot.png">

##### Run lowess using Staten Island AIS sample (Pre-processed)

In [18]:
lowess = sm.nonparametric.lowess
lowess_ais = lowess(ais_sdf['latitude'], ais_sdf['longitude'])

#### Plot Lowess first run

In [ ]:
plotly.offline.plot({
    "data": [plotly.graph_objs.Scatter(x=lowess_ais[:,0], y=lowess_ais[:,1])]
})

<img src="img/newplot1.png">

#### Pre-QC Route Layer on WebMap

In [ ]:
preqc_route_layer = gis.content.search("AIS_SIFerryRoute_PreQC", item_type="feature service")[0]
preqc_route_layer

In [ ]:
preqc_route_map = gis.map("New York City")
# preqc_route_map.basemap = 'ocean'
# preqc_route_map.add_layer(preqc_route_layer)
preqc_route_map

In [ ]:
preqc_route_map.basemap = 'gray'

In [ ]:
preqc_route_map.add_layer(ais_sample)

In [ ]:
preqc_route_map.add_layer(preqc_route_layer)

<img src="img/capture01.PNG">

#### The north end of the route overshoots the terminal... let's fix this.

# Part 3: Run QC - Density-based Clustering

#### Explanation of Density-based Clustering

http://pro.arcgis.com/en/pro-app/tool-reference/spatial-statistics/how-density-based-clustering-works.htm

<img src="img/capture02.PNG">

<img src="img/capture03.PNG">

#### Code to Retrieve Density-based Clustering Layers (with noise, and without noise) and Show on Maps

DBSCAN With noise

In [ ]:
dbcluster_lyr_full = gis.content.search("OPTICS_AIS_Sampler_50ft_lowsens")[0]
dbcluster_map = gis.map("New York City")
dbcluster_map.basemap = "gray"
dbcluster_map.add_layer(dbcluster_lyr_full)
dbcluster_map

<img src="img/capture04.PNG">

Without noise

In [ ]:
dbcluster_lyr_full_nonoise = gis.content.search("OPTICS_AIS_Sampler_50ft_lowsens_FILTER_nonnoise")[0]
dbcluster_map = gis.map("New York City")
dbcluster_map.basemap = "gray"
dbcluster_map.add_layer(dbcluster_lyr_full_nonoise)
dbcluster_map

<img src="img/capture.png">

# Part 4: Post QC - Map

#### Lowess Layer on WebMap

In [ ]:
postqc_route_layer = gis.content.search("AIS_SIFerryRoute_PostQC")[0]
postqc_route_layer

In [ ]:
postqc_route_map = gis.map("Staten Island")
# preqc_route_map.basemap = 'ocean'
# preqc_route_map.add_layer(preqc_route_layer)
postqc_route_map

In [ ]:
postqc_route_map.basemap = 'gray'

In [ ]:
postqc_route_map.add_layer(postqc_route_layer)

<img src="img/capture06.png">

### Next Steps

- Continued improvement on generalization function. 
- Iteratation on multiple routes.  
- Output lowess line into geospatial web application. 
- Published authoritative records into <a ref="https://osav-usdot.opendata.arcgis.com/">Bureau of Transportation Open Data site.</a>